In [1]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import random
import os,pdb
from torch import optim
from torch.autograd import Variable
from PIL import Image


In [2]:
# Load Data
def train_load(path, parts,mode='train'):
    
    images = []
    labels = []

    for p in parts:
        npz = np.load(os.path.join(path, str(p) + ".npz"))

        images.append(npz['arr_0'])
        labels.append(npz['arr_1'])
        

    images = np.concatenate(images)
    labels = np.concatenate(labels)
    labels = np.asarray([1 if x in [330.0,331.0] else 0 for x in labels])
    
    #Balance data
    cancer = np.count_nonzero(labels)
    noncancer = (labels.shape[0]-cancer)
    minimum = min(cancer,noncancer)
    sample_idxs_cancer = random.sample(list(np.where(labels == 1)[0]), minimum)
    sample_idxs_nocancer = random.sample(list(np.where(labels == 0)[0]), minimum)
    
    if mode=='dev':
        sample_idxs_cancer = sample_idxs_cancer[:1000]
        sample_idxs_nocancer = sample_idxs_nocancer[:1000] 
    
    new_idxs = []
    new_idxs.extend(sample_idxs_cancer)
    new_idxs.extend(sample_idxs_nocancer)
    random.shuffle(new_idxs)
    images = images[new_idxs]
    labels = labels[new_idxs]
    
    pdb.set_trace()
    print("\nLoaded", len(images), "patches ")
    
    return images,labels

train_dataset_path = '/home/ubuntu/codebase/Semi-Supervised-GANs/dataset/patch_data/train'
dev_dataset_path = '/home/ubuntu/codebase/Semi-Supervised-GANs/dataset/patch_data/dev'

trainX,trainY = train_load(train_dataset_path,range(5,11))
devX,devY = train_load(dev_dataset_path,range(4,5),mode='dev')


> <ipython-input-2-9e66dd1fd7f0>(37)train_load()
-> print("\nLoaded", len(images), "patches ")
(Pdb) c

Loaded 41000 patches 
> <ipython-input-2-9e66dd1fd7f0>(37)train_load()
-> print("\nLoaded", len(images), "patches ")
(Pdb) c

Loaded 2000 patches 


In [3]:
class make_dataset(Dataset):

    def __init__(self, X, Y, transform=None, mode='train'):
        self.patches = X
        self.labels = Y
        self.transform = transform
        self.mode = mode

    def __len__(self):
        return self.labels.shape[0]

    def __getitem__(self, idx):
        if self.mode == 'train':                
            return self.transform(Image.fromarray(self.patches[idx])),self.labels[idx]
        
        else:
            
            return self.transform(Image.fromarray(self.patches[idx]))

In [4]:
# Set Defaults
num_epochs = 10 
batch_size = 32
num_classes = 10 # number of classes for dataset
lr = 0.0002 
b1 = 0.5 # adam: decay of first order momentum of gradient
b2 = 0.999 # adam: decay of first order momentum of gradient
n_cpu = 8 # number of cpu threads to use during batch generation
latent_dim = 100 # dimensionality of the latent space
img_size = 32 # size of each image dimension
channels = 1 # number of output image channels
sample_interval = 400 # interval between image sampling

In [5]:
# Set cuda
if torch.cuda.is_available():
    cuda = True 
else:
    cuda = False

In [6]:
def weights_init_normal(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.zero_()

In [7]:
class Model(nn.Module):
    
    def __init__(self):
        super(Model, self).__init__()
        self.use_gpu = cuda
        drop_rate = 0.20
        num_classes = 2
        input_channels = 3
        ndf = 16
        
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels=input_channels, out_channels=ndf, kernel_size=3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(ndf),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout2d(drop_rate),
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(in_channels=ndf, out_channels=ndf*2, kernel_size=3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(ndf*2),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout2d(drop_rate),
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(in_channels=ndf*2, out_channels=ndf*4, kernel_size=3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(ndf*4),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout2d(drop_rate),
        )
        self.conv4 = nn.Sequential(
            nn.Conv2d(in_channels=ndf*4, out_channels=ndf*8, kernel_size=3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(ndf*8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout2d(drop_rate),
        )
        
        # The height and width of downsampled image
        ds_size = img_size // 2**4
        
        # Output layer
        self.linear1 = nn.Sequential(
            nn.Linear(in_features=128*ds_size**2, out_features=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        
        # Flatten the features to give as input to the fc linear classification layer
        out = x.view(x.shape[0], -1)
        out = self.linear1(out)
        
        return out

In [8]:
# Initialize loss and model
loss = nn.BCELoss()
model = Model()

if cuda:
    loss.cuda()
    model = nn.DataParallel(model)
    model.cuda()

In [9]:
# Initialize weights
model.apply(weights_init_normal)
optimizer = optimizer = torch.optim.Adam(model.parameters(), lr=lr,betas=(b1, b2))

In [10]:
transformations = transforms.Compose([
    transforms.Scale(32),
    transforms.ToTensor(),
    transforms.Normalize((.5, .5, .5), (.5, .5, .5)),
    ])

/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torchvision-0.2.1-py3.6.egg/torchvision/transforms/transforms.py:188: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.


In [11]:
training_data = make_dataset(trainX,trainY, transformations)
train_loader = DataLoader(training_data, batch_size=batch_size, shuffle=True)
dev_data = make_dataset(devX,devY, transformations)
dev_loader = DataLoader(dev_data, batch_size=batch_size, shuffle=False)

In [12]:
# Training Function
def train(optimizer, criterion, dataloader, model):
    model.train()
    
    epoch_loss = 0

    for i, (imgs, labels) in enumerate(dataloader):
        
        imgs = Variable(imgs.float())
        labels = Variable(labels.float())
        #print(labels)
        if cuda:
            imgs = imgs.cuda()
            labels = labels.cuda()

        optimizer.zero_grad()

        outputs = model(imgs)
        loss = criterion(outputs, labels)
        epoch_loss += loss.item()
        loss.backward()
        optimizer.step()
    
    return np.mean(epoch_loss)

In [13]:
# Testing Function
def test(criterion, dataloader, model, mode = None):
    model.eval()
    
    epoch_loss = 0
    total = 0
    correct = 0
    
    for i, (imgs, labels) in enumerate(dataloader):
        
        imgs = imgs.float()
        labels = labels.float()
        
        if cuda:
            imgs = imgs.cuda()
            labels = labels.cuda()

        outputs = model(imgs)
        loss = criterion(outputs, labels)
        epoch_loss += loss.item()

        if mode =='test':
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels.long()).sum()
                
    epoch_loss = np.mean(epoch_loss)
    
    if mode == 'test':
        test_accuracy = 100 * correct / total
        return epoch_loss,test_accuracy
    
    return epoch_loss

In [14]:
for epoch in range(num_epochs):
    train_loss = train(optimizer, loss, train_loader, model)
    valid_loss = test(loss, dev_loader, model)
    test_loss, test_accuracy = test(loss, dev_loader, model,mode='test')
    
    # Display Progress
    print ("[Epoch %d/%d], Train Loss = %f, Validation Loss = %f, Test Loss = %f, Test Accuracy = %f" % (epoch, num_epochs, train_loss, 
                                                                                    valid_loss, test_loss, test_accuracy))
    

/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torch/nn/functional.py:1594: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))
/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torch/nn/functional.py:1594: UserWarning: Using a target size (torch.Size([8])) that is different to the input size (torch.Size([8, 1])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))
/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torch/nn/functional.py:1594: UserWarning: Using a target size (torch.Size([16])) that is different to the input size (torch.Size([16, 1])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.si

[Epoch 0/10], Train Loss = 725.466133, Validation Loss = 49.491781, Test Loss = 49.491781, Test Accuracy = 50.000000


KeyboardInterrupt: 